# 🐍Python — Data Preparation & Business EDA

In [4]:
import pandas as pd
import numpy as np
import sqlite3

conn = sqlite3.connect(":memory:")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


# ✅ Load the Data

In [2]:
accounts = pd.read_csv("/content/ravenstack_accounts.csv")
subscriptions = pd.read_csv("/content/ravenstack_subscriptions.csv")
feature_usage = pd.read_csv("/content/ravenstack_feature_usage.csv")
support_tickets = pd.read_csv("/content/ravenstack_support_tickets.csv")
churn_events = pd.read_csv("/content/ravenstack_churn_events.csv")


# ✅ Sanity Check: Shape & Columns

In [3]:
datasets = {
    "accounts": accounts,
    "subscriptions": subscriptions,
    "feature_usage": feature_usage,
    "support_tickets": support_tickets,
    "churn_events": churn_events
}

for name, df in datasets.items():
    print(f"\n{name.upper()}")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))




ACCOUNTS
Shape: (500, 10)
Columns: ['account_id', 'account_name', 'industry', 'country', 'signup_date', 'referral_source', 'plan_tier', 'seats', 'is_trial', 'churn_flag']

SUBSCRIPTIONS
Shape: (5000, 14)
Columns: ['subscription_id', 'account_id', 'start_date', 'end_date', 'plan_tier', 'seats', 'mrr_amount', 'arr_amount', 'is_trial', 'upgrade_flag', 'downgrade_flag', 'churn_flag', 'billing_frequency', 'auto_renew_flag']

FEATURE_USAGE
Shape: (25000, 8)
Columns: ['usage_id', 'subscription_id', 'usage_date', 'feature_name', 'usage_count', 'usage_duration_secs', 'error_count', 'is_beta_feature']

SUPPORT_TICKETS
Shape: (2000, 9)
Columns: ['ticket_id', 'account_id', 'submitted_at', 'closed_at', 'resolution_time_hours', 'priority', 'first_response_time_minutes', 'satisfaction_score', 'escalation_flag']

CHURN_EVENTS
Shape: (600, 9)
Columns: ['churn_event_id', 'account_id', 'churn_date', 'reason_code', 'refund_amount_usd', 'preceding_upgrade_flag', 'preceding_downgrade_flag', 'is_reactivatio

# ✅ Convert Date Columns

In [5]:
# Accounts
accounts['signup_date'] = pd.to_datetime(accounts['signup_date'])

# Subscriptions
subscriptions['start_date'] = pd.to_datetime(subscriptions['start_date'])
subscriptions['end_date'] = pd.to_datetime(subscriptions['end_date'], errors='coerce')

# Feature usage
feature_usage['usage_date'] = pd.to_datetime(feature_usage['usage_date'])

# Support tickets
support_tickets['submitted_at'] = pd.to_datetime(support_tickets['submitted_at'])
support_tickets['closed_at'] = pd.to_datetime(support_tickets['closed_at'])

# Churn events
churn_events['churn_date'] = pd.to_datetime(churn_events['churn_date'])


# ✅ Validate Data Types

In [10]:
accounts.dtypes
# subscriptions.dtypes
# feature_usage.dtypes
# support_tickets.dtypes
# churn_events.dtypes



,0
account_id,object
account_name,object
industry,object
country,object
signup_date,datetime64[ns]
referral_source,object
plan_tier,object
seats,int64
is_trial,bool
churn_flag,bool


# ✅ Create Account-Level Churn Flag

In [11]:
accounts['is_churned'] = accounts['account_id'].isin(churn_events['account_id'])
accounts['is_churned'] = accounts['is_churned'].astype(int)


In [14]:
accounts["is_churned"].head()

,is_churned
0,1
1,0
2,1
3,1
4,1


# 🧮 Account Tenure (in days)

In [19]:
churn_dates = (
    churn_events
    .groupby('account_id', as_index=False)['churn_date']
    .min()
)



In [20]:
accounts = accounts.merge(churn_dates, on='account_id', how='left')


In [21]:
analysis_date = churn_dates['churn_date'].max()


In [22]:
accounts['tenure_days'] = np.where(
    accounts['churn_date'].notna(),
    (accounts['churn_date'] - accounts['signup_date']).dt.days,
    (analysis_date - accounts['signup_date']).dt.days
)


# 🧮 Tenure Buckets (Very Important for Churn Timing)

In [23]:
accounts['tenure_bucket'] = pd.cut(
    accounts['tenure_days'],
    bins=[0, 30, 90, 180, 365, np.inf],
    labels=['0–30 days', '31–90 days', '91–180 days', '181–365 days', '365+ days']
)


In [24]:
accounts[['account_id', 'signup_date', 'churn_date', 'tenure_days']].head()


,account_id,signup_date,churn_date,tenure_days
0,A-2e4581,2024-10-16,2024-11-23,38.00
1,A-43a9e3,2023-08-17,NaT,502.00
2,A-0a282f,2024-08-27,2024-10-06,40.00
3,A-1f0ac7,2023-08-27,2024-11-08,439.00
4,A-ce550d,2024-10-27,2024-12-28,62.00


In [25]:
accounts['tenure_days'].describe()


,tenure_days
count,500.00
mean,193.92
std,191.55
min,0.00
25%,39.75
50%,123.50
75%,297.25
max,729.00


# Load CSVs into SQL Tables

In [27]:
accounts.to_sql("accounts", conn, index=False, if_exists="replace")
subscriptions.to_sql("subscriptions", conn, index=False, if_exists="replace")
feature_usage.to_sql("feature_usage", conn, index=False, if_exists="replace")
support_tickets.to_sql("support_tickets", conn, index=False, if_exists="replace")
churn_events.to_sql("churn_events", conn, index=False, if_exists="replace")


600

In [28]:
query = """
SELECT
    tenure_bucket,
    COUNT(*) AS total_accounts,
    SUM(is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(is_churned) / COUNT(*), 3) AS churn_rate
FROM accounts
GROUP BY tenure_bucket
ORDER BY churn_rate DESC;
"""

pd.read_sql(query, conn)


,tenure_bucket,total_accounts,churned_accounts,churn_rate
0,0–30 days,93,88,0.95
1,31–90 days,117,97,0.83
2,None,4,3,0.75
3,91–180 days,89,66,0.74
4,181–365 days,104,69,0.66
5,365+ days,93,29,0.31


In [29]:
query = """
SELECT
    plan_tier,
    COUNT(*) AS total_accounts,
    SUM(is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(is_churned) / COUNT(*), 3) AS churn_rate
FROM accounts
GROUP BY plan_tier
ORDER BY churn_rate DESC;
"""

pd.read_sql(query, conn)


,plan_tier,total_accounts,churned_accounts,churn_rate
0,Pro,178,129,0.72
1,Enterprise,154,108,0.70
2,Basic,168,115,0.69


In [30]:
query = """
SELECT
    is_trial,
    COUNT(*) AS total_accounts,
    SUM(is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(is_churned) / COUNT(*), 3) AS churn_rate
FROM accounts
GROUP BY is_trial;
"""

pd.read_sql(query, conn)


,is_trial,total_accounts,churned_accounts,churn_rate
0,0,403,285,0.71
1,1,97,67,0.69



# 1️⃣ Engagement vs Churn
* Are low-engagement customers more likely to churn?



In [31]:
query = """WITH usage_per_account AS (
    SELECT
        s.account_id,
        SUM(fu.usage_count) AS total_usage
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    CASE
        WHEN total_usage < 100 THEN 'Low Engagement'
        WHEN total_usage BETWEEN 100 AND 500 THEN 'Medium Engagement'
        ELSE 'High Engagement'
    END AS engagement_level,
    COUNT(*) AS total_accounts,
    SUM(a.is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(a.is_churned) / COUNT(*), 3) AS churn_rate
FROM usage_per_account u
JOIN accounts a
    ON u.account_id = a.account_id
GROUP BY engagement_level
ORDER BY churn_rate DESC; """

pd.read_sql_query(query,conn)



,engagement_level,total_accounts,churned_accounts,churn_rate
0,Low Engagement,1,1,1.00
1,High Engagement,248,175,0.71
2,Medium Engagement,251,176,0.70


# 2️⃣ Early Warning Signal
* Does usage drop before churn? (last 30 days)

In [32]:
query = """WITH last_usage AS (
    SELECT
        s.account_id,
        MAX(fu.usage_date) AS last_usage_date
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    a.is_churned,
    ROUND(AVG(julianday(a.churn_date) - julianday(l.last_usage_date)), 1)
        AS avg_days_since_last_usage
FROM accounts a
LEFT JOIN last_usage l
    ON a.account_id = l.account_id
WHERE a.churn_date IS NOT NULL
GROUP BY a.is_churned; """

pd.read_sql_query(query,conn)


,is_churned,avg_days_since_last_usage
0,1,-189.60


# 3️⃣ Inactivity Risk
* Does longer inactivity increase churn risk?

In [33]:
query ="""WITH inactivity AS (
    SELECT
        s.account_id,
        julianday(MAX(fu.usage_date)) AS last_usage_jd
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    CASE
        WHEN (julianday(a.churn_date) - last_usage_jd) <= 7 THEN '0–7 days'
        WHEN (julianday(a.churn_date) - last_usage_jd) <= 30 THEN '8–30 days'
        ELSE '30+ days'
    END AS inactivity_bucket,
    COUNT(*) AS total_accounts,
    SUM(a.is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(a.is_churned) / COUNT(*), 3) AS churn_rate
FROM accounts a
JOIN inactivity i
    ON a.account_id = i.account_id
WHERE a.churn_date IS NOT NULL
GROUP BY inactivity_bucket
ORDER BY churn_rate DESC; """

pd.read_sql_query(query,conn)

,inactivity_bucket,total_accounts,churned_accounts,churn_rate
0,8–30 days,11,11,1.00
1,30+ days,5,5,1.00
2,0–7 days,336,336,1.00


# 4️⃣ Product Quality Signal
* Do higher error rates increase churn?

In [34]:
query="""WITH error_metrics AS (
    SELECT
        s.account_id,
        SUM(fu.error_count) AS total_errors
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    CASE
        WHEN total_errors = 0 THEN 'No Errors'
        WHEN total_errors BETWEEN 1 AND 10 THEN 'Low Errors'
        ELSE 'High Errors'
    END AS error_level,
    COUNT(*) AS total_accounts,
    SUM(a.is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(a.is_churned) / COUNT(*), 3) AS churn_rate
FROM error_metrics e
JOIN accounts a
    ON e.account_id = a.account_id
GROUP BY error_level
ORDER BY churn_rate DESC; """

pd.read_sql_query(query,conn)


,error_level,total_accounts,churned_accounts,churn_rate
0,No Errors,1,1,1.00
1,High Errors,477,336,0.70
2,Low Errors,22,15,0.68


# 5️⃣ Feature Adoption Depth
* Do customers using more features churn less?

In [35]:
query= """WITH feature_depth AS (
    SELECT
        s.account_id,
        COUNT(DISTINCT fu.feature_name) AS features_used
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    CASE
        WHEN features_used <= 3 THEN 'Low Adoption'
        WHEN features_used BETWEEN 4 AND 10 THEN 'Medium Adoption'
        ELSE 'High Adoption'
    END AS adoption_level,
    COUNT(*) AS total_accounts,
    SUM(a.is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(a.is_churned) / COUNT(*), 3) AS churn_rate
FROM feature_depth f
JOIN accounts a
    ON f.account_id = a.account_id
GROUP BY adoption_level
ORDER BY churn_rate DESC; """

pd.read_sql_query(query,conn)


,adoption_level,total_accounts,churned_accounts,churn_rate
0,Medium Adoption,1,1,1.00
1,High Adoption,499,351,0.70


#6️⃣ Beta Feature Risk
* Does beta feature usage impact churn?

In [36]:
query="""WITH beta_usage AS (
    SELECT
        s.account_id,
        MAX(fu.is_beta_feature) AS used_beta
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    used_beta,
    COUNT(*) AS total_accounts,
    SUM(a.is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(a.is_churned) / COUNT(*), 3) AS churn_rate
FROM beta_usage b
JOIN accounts a
    ON b.account_id = a.account_id
GROUP BY used_beta; """

pd.read_sql_query(query,conn)


,used_beta,total_accounts,churned_accounts,churn_rate
0,0,8,3,0.38
1,1,492,349,0.71


# 7️⃣ Lifecycle Sensitivity
* Is usage more important early vs late tenure?

In [37]:
query="""WITH usage_totals AS (
    SELECT
        s.account_id,
        SUM(fu.usage_count) AS total_usage
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    a.tenure_bucket,
    CASE
        WHEN u.total_usage < 100 THEN 'Low Usage'
        ELSE 'High Usage'
    END AS usage_level,
    ROUND(AVG(a.is_churned), 3) AS churn_rate
FROM accounts a
JOIN usage_totals u
    ON a.account_id = u.account_id
GROUP BY a.tenure_bucket, usage_level
ORDER BY a.tenure_bucket, churn_rate DESC; """

pd.read_sql_query(query,conn)


,tenure_bucket,usage_level,churn_rate
0,None,High Usage,0.75
1,0–30 days,High Usage,0.95
2,181–365 days,High Usage,0.66
3,31–90 days,Low Usage,1.00
4,31–90 days,High Usage,0.83
5,365+ days,High Usage,0.31
6,91–180 days,High Usage,0.74


# 8️⃣ Risk Segmentation
* Create churn-risk groups using usage + errors

In [38]:
query =""" WITH metrics AS (
    SELECT
        s.account_id,
        SUM(fu.usage_count) AS usage_cnt,
        SUM(fu.error_count) AS error_cnt
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    CASE
        WHEN usage_cnt < 100 AND error_cnt > 10 THEN 'High Risk'
        WHEN usage_cnt < 200 THEN 'Medium Risk'
        ELSE 'Low Risk'
    END AS churn_risk_segment,
    COUNT(*) AS total_accounts,
    SUM(a.is_churned) AS churned_accounts,
    ROUND(1.0 * SUM(a.is_churned) / COUNT(*), 3) AS churn_rate
FROM metrics m
JOIN accounts a
    ON m.account_id = a.account_id
GROUP BY churn_risk_segment
ORDER BY churn_rate DESC; """

pd.read_sql_query(query,conn)

,churn_risk_segment,total_accounts,churned_accounts,churn_rate
0,Medium Risk,17,12,0.71
1,Low Risk,483,340,0.70


# 9️⃣ Intervention Threshold

* Where should the business intervene?



In [39]:
query= """SELECT
    CASE
        WHEN total_usage < 50 THEN 'Critical – Immediate Action'
        WHEN total_usage < 150 THEN 'Warning – Nurture'
        ELSE 'Healthy'
    END AS intervention_group,
    COUNT(*) AS accounts,
    ROUND(AVG(is_churned), 3) AS churn_rate
FROM (
    SELECT
        a.account_id,
        a.is_churned,
        SUM(fu.usage_count) AS total_usage
    FROM accounts a
    JOIN subscriptions s ON a.account_id = s.account_id
    JOIN feature_usage fu ON s.subscription_id = fu.subscription_id
    GROUP BY a.account_id
)
GROUP BY intervention_group; """

pd.read_sql_query(query,conn)


,intervention_group,accounts,churn_rate
0,Healthy,495,0.70
1,Warning – Nurture,5,0.80


# EXPORT FOR POWER BI

In [40]:
usage_metrics_account = pd.read_sql("""
SELECT
    s.account_id,
    SUM(fu.usage_count) AS total_usage,
    COUNT(DISTINCT fu.feature_name) AS features_used,
    SUM(fu.error_count) AS total_errors,
    MAX(fu.usage_date) AS last_usage_date
FROM feature_usage fu
JOIN subscriptions s
    ON fu.subscription_id = s.subscription_id
GROUP BY s.account_id
""", conn)


In [41]:
usage_metrics_account['last_usage_date'] = pd.to_datetime(
    usage_metrics_account['last_usage_date']
)

usage_metrics_account['days_since_last_usage'] = (
    analysis_date - usage_metrics_account['last_usage_date']
).dt.days


In [42]:
support_metrics_account = pd.read_sql("""
SELECT
    account_id,
    COUNT(ticket_id) AS total_tickets,
    AVG(resolution_time_hours) AS avg_resolution_hours,
    AVG(first_response_time_minutes) AS avg_first_response_minutes,
    SUM(escalation_flag) AS escalation_count
FROM support_tickets
GROUP BY account_id
""", conn)


In [43]:
churn_risk_segments = pd.read_sql("""
WITH metrics AS (
    SELECT
        s.account_id,
        SUM(fu.usage_count) AS usage_cnt,
        SUM(fu.error_count) AS error_cnt
    FROM feature_usage fu
    JOIN subscriptions s
        ON fu.subscription_id = s.subscription_id
    GROUP BY s.account_id
)
SELECT
    m.account_id,
    CASE
        WHEN usage_cnt < 100 AND error_cnt > 10 THEN 'High Risk'
        WHEN usage_cnt < 200 THEN 'Medium Risk'
        ELSE 'Low Risk'
    END AS churn_risk_segment
FROM metrics m
""", conn)


In [44]:
accounts_final = accounts[[
    'account_id',
    'industry',
    'country',
    'plan_tier',
    'is_trial',
    'tenure_days',
    'tenure_bucket',
    'is_churned'
]]


In [47]:
import os

# Create the directory if it doesn't exist
os.makedirs('powerbi_exports', exist_ok=True)

accounts_final.to_csv("powerbi_exports/accounts_final.csv", index=False)
usage_metrics_account.to_csv("powerbi_exports/usage_metrics_account.csv", index=False)
support_metrics_account.to_csv("powerbi_exports/support_metrics_account.csv", index=False)
churn_risk_segments.to_csv("powerbi_exports/churn_risk_segments.csv", index=False)